## The 10-line CTA

This strategy is based on the article https://www.linkedin.com/pulse/implement-cta-less-than-10-lines-code-thomas-schmelzer/

In [14]:
import pandas as pd
import numpy as np
import beakerx as bx

In [2]:
# load data
fut = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before=pd.Timestamp("1990-01-01"))

In [6]:
# compute volatility adjusted returns and winsorize (e.g. clip) them. 
volatility = np.log(fut).diff().ewm(com=32).std()

In [7]:
# move back into a "price"-space by accumulating filtered log returns
prices = (np.log(fut).diff() / volatility).clip(-4.2, 4.2).cumsum()

In [8]:
# simple oscillator function (apply some rescaling to enforce constant norm for N(0,1) returns)
def osc(prices, fast=32, slow=96):
    f,g = 1 - 1/fast, 1-1/slow
    return (prices.ewm(span=2*fast-1).mean() - prices.ewm(span=2*slow-1).mean())/np.sqrt(1.0 / (1 - f * f) - 2.0 / (1 - f * g) + 1.0 / (1 - g * g))


In [9]:
# compute the currency position and apply again some winsorizing to avoid extremely large positions, strictly speaking volatility of percent returns needed here...
CurrencyPosition = (50000*np.tanh(osc(prices, fast=16, slow=48)) / volatility).clip(-5e7, 5e7)


In [10]:
# the profit today is the return today times the position of yesterday
Profit = (fut.pct_change() * CurrencyPosition.shift(periods=1)).sum(axis=1)

In [16]:
# simulate the compounding over time
nav = (1 + Profit / 7e7).cumprod().to_frame("nav") #.plot(logy=True, grid=True)
bx.SimpleTimePlot(nav, ['nav'], logY=True)